In [58]:
### generate data
import numpy as np
np.random.seed(5)

def generate_data(n_samples, true_coefficients, noise_level=0.1):

    # Generate a random input signal
    k = len(true_coefficients)
    m, n = true_coefficients[0].shape

    x = np.random.randn(n_samples, m)

    # Generate the desired signal by passing the input through the known system
    y = np.zeros((n_samples, n)) 

    for j in range(k):
        y += np.dot(x, true_coefficients[j]) 
    y += noise_level * np.random.randn(n_samples, n)
    
    return x, y

# Example usage
n_samples = 300
# a series of matrix: k x m x n 
# k : influence scope 5
# m : output dimension  3
# n : input dimension 2

k = 1
m = 3
n = 2
true_coefficients = [np.random.randn(m,n) for _ in range(k)]
noise_level = 0.1
x, y = generate_data(n_samples, true_coefficients, noise_level)
print("Input signal:", x[:3])
print("Desired signal:", y[:3])

Input signal: [[-0.9092324  -0.59163666  0.18760323]
 [-0.32986996 -1.19276461 -0.20487651]
 [-0.35882895  0.6034716  -1.66478853]]
Desired signal: [[-1.83238084  0.68710999]
 [-3.04669364  0.08595731]
 [ 1.18611853 -2.76397773]]


In [59]:
### using kalman filter to estimate the coefficients
def kalman_filter(x, y, k):
    n_samples = len(y)
    m = x.shape[1]
    n = y.shape[1]
    
    # Initialize the state estimate
    # Initialize w: k x m x n randomly
    w = np.ones((k, m, n)) * 0.1

    # PQR matrices, initialized as identity matrices, each one is k x n x n
    P = np.array([np.eye(m*n) for _ in range(k)]) * 1
    # Q = np.array([np.eye(m*n) for _ in range(k)]) * 0.01
    R = np.array([np.eye(n) for _ in range(k)]) * 1
    
    Ws = [] # record the Ws
    for i in range(k, n_samples):
        # Predict
        
        for j in range(k):
            ## w[j] add some samll noise
            # w[j] = w[j] +  np.random.randn(m, n) * 0.01
            # P[j] = P[j] 
            pass 
        
        # Update
        
        ### error 
        y_hat = np.zeros(n)
        for j in range(k):
            y_hat += np.dot(x[i-j], w[j])
        
        
        for j in range(k):
            ## w[j] add K[j] * e
            W_temp = w[j].T.reshape(-1,1) 
            H_temp = np.zeros((n, m*n))
            for l in range(n):
                H_temp[l, m*l:m*(l+1)] = x[i-j] 
            S = np.dot(H_temp, np.dot(P[j], H_temp.T)) + R[j]
            K = np.dot(P[j], np.dot(H_temp.T, np.linalg.inv(S))) 

            e = y[i] - y_hat
            W_temp += np.dot(K.reshape(m*n, n), e.reshape(-1,1)) 
            w[j] = W_temp.reshape(n, m).T

            P[j] = np.dot((np.eye(m*n) - np.dot(K, H_temp)), P[j]) ### P is reasonable 
            Ws.append(w[j].copy())
        # if i % 5 == 0:
        #     print(w[0])
            
    return w, Ws

In [60]:
## Example usage

estimated_coefficients, Ws = kalman_filter(x, y, k)
print("True coefficients:", true_coefficients)
print("Estimated coefficients:", estimated_coefficients)

True coefficients: [array([[ 0.44122749, -0.33087015],
       [ 2.43077119, -0.25209213],
       [ 0.10960984,  1.58248112]])]
Estimated coefficients: [[[ 0.43854466 -0.34085512]
  [ 2.44063378 -0.24298213]
  [ 0.09980496  1.57500669]]]


In [61]:
### save Ws to file
import pickle
with open("NoMemKFWs.pkl", "wb") as f:
    pickle.dump(Ws, f)


In [62]:
Ws[0].shape

(3, 2)